In [2]:
from dataclasses import dataclass
import asyncio
import typing as t
from functools import cached_property

T = t.TypeVar('T')

@dataclass
class accessor(t.Generic[T]):
    set: t.Callable[[], t.Coroutine[t.Any, t.Any, T]]
    get: t.Callable[[T], t.Coroutine[t.Any, t.Any, None]]


class awaitable_property(cached_property, t.Generic[T]):
    """

    """

    def __init__(self: 'awaitable_property[T]',
                 fget: t.Optional[t.Callable[[t.Any], T]] = ...,
                 fset: t.Optional[t.Callable[[t.Any, T], None]] = ...,
                 fdel: t.Optional[t.Callable[[t.Any], None]] = ...,
                 doc: t.Optional[str] = ...) -> None:
        self.fget = fget
        self.fset = fset
        self.fdel = fdel
        if doc is None and fget is not None:
            doc = fget.__doc__
        self.__doc__ = doc
        super().__init__(self._create_accessor)

    @t.overload
    def __get__(self: 'awaitable_property[T]', obj: None, kls: type = ...) -> 'awaitable_property[T]':
        ...

    @t.overload
    def __get__(self: 'awaitable_property[T]', obj: t.Any, kls: None = ...) -> accessor[T]:
        ...

    def __get__(self: 'awaitable_property[T]', obj: t.Optional[t.Any] = None, kls: t.Optional[type] = None):
        return super().__get__(obj, kls)

    def __set__(self, obj, value):
        raise AttributeError(f"can't set attribute directly, use set()")

    def _create_accessor(self, obj):
        condition = asyncio.Condition()
        return accessor(
            get=lambda: self._get(condition=condition, obj=obj),
            set=lambda value: self._set(condition=condition, obj=obj, value=value)
        )

    async def _set(self, *, condition: asyncio.Condition, obj: t.Any, value):
        if self.fset is None:
            raise AttributeError(f"can't set attribute {self.attrname}")
        async with condition:
            self.fset(obj, value)
            condition.notify_all()

    async def _get(self, *, condition: asyncio.Condition, obj: t.Any):
        if self.fget is None:
            raise AttributeError(f'unreadable attribute {self.attrname}')
        async with condition:
            await condition.wait()
            return self.fget(obj)

    def getter(self, fget):
        prop = type(self)(fget, self.fset, self.fdel, self.__doc__)
        prop.attrname = self.attrname
        return prop

    def setter(self, fset):
        prop = type(self)(self.fget, fset, self.fdel, self.__doc__)
        prop.attrname = self.attrname
        return prop

    def deleter(self, fdel):
        prop = type(self)(self.fget, self.fset, fdel, self.__doc__)
        prop.attrname = self.attrname
        return prop


In [3]:
class A:
    value = 0

    @awaitable_property
    def blub(self):
        return self.value

    @blub.setter
    def blub(self, value):
        self.value = value

In [4]:
x = A()


async def get_blub_safe():
    return await x.blub.get()


async def write_blub_safe():
    await x.blub.set(5)


await asyncio.gather(get_blub_safe(), write_blub_safe())

[5, None]

In [5]:
x.blub = 4
await x.blub.set(4)

AttributeError: can't set attribute directly, use set()

In [ ]:
await x.blub.get()

In [ ]:
await x.blub.set(4)